# Usuage example for the beach pedestrian detector

Authors:
- Guillem Bibiloni Femenias
- Joan Josep Ordóñez Bonet

In this file it is shown an example of how the detector can be used. Initially, the data has to be loaded using the function ```load_data``` from the module, indicating the folder from where the images come from. This folder must contain also the labeling of the images. After this load, the proper execution of the detector can be run for each image using the ```analyze_image``` function, returning both an image with the regions considered as pedestrians and the centroid of each region, where the person is supposed to be at.

After this analysis, a proper evaluation can be performed, both for the entire dataset and for each image. The evaluation on the whole dataset is performed by analysing the _MSE_ metric on the count of detections with the ```evaluate_mse``` function. The evaluation on each image is performed by using the accuracy of the centroids detected with the ```evaluate_accuracy``` function. This evaluation is parametrized with a threshold, which is used as the maximum euclidean distance to which a detection is considered valid.

In [1]:
import image_analyzer
import cv2

color_images, gt_positions, gt_count = image_analyzer.load_data("images")

results = []
centroids = []

for image in color_images:
    resulting_img, detections = image_analyzer.analyze_image(image)
    results.append(resulting_img)
    centroids.append(detections)
    
for image, name, cents in zip(results, gt_count["Image"], centroids):
    cv2.imwrite("results/{}".format(name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    
    for c in cents:
        cv2.circle(image, c, 5, (0,255,0), -1)

    cv2.imwrite("results_centroids/{}".format(name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

detector_mse = image_analyzer.evaluate_mse(gt_count["NPeople"], [len(x) for x in centroids])

threshold = 15
scores = []

for idx, image in enumerate(gt_count["Image"].tolist()):
    predict_ = centroids[idx]
    gt_ = gt_positions[gt_positions["Image"] == image][["X", "Y"]].values.tolist()

    scores.append(image_analyzer.evaluate_accuracy(gt_, predict_, threshold))

For this dataset, the results are as follows:

- Overall MSE metric:

In [2]:
detector_mse

2280.4

- Accuracies:

In [3]:
scores

[0,
 0,
 0.34285714285714286,
 0.38461538461538464,
 0.5081967213114754,
 0.524390243902439,
 0.5411764705882353,
 0.40425531914893614,
 0.46,
 0.3894736842105263]